In [4]:
import random
import glob
import shutil
import os

# 지정된 경로에서 .json 파일 리스트 가져오기
file_list = glob.glob('../data//train/train_*.json')

# 랜덤하게 7000개 파일 선택
selected_files = random.sample(file_list, 7000)

# 파일 이름 오름차순으로 정렬
selected_files.sort()

# 특정 디렉토리 생성 (이미 존재하는 경우에는 패스)
target_directory = "../data/selected_train/"
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# 선택된 파일을 특정 디렉토리에 복사하고 이름 변경
for i, file in enumerate(selected_files):
    new_filename = f"{target_directory}train_{100000 + i}.json"
    shutil.copy(file, new_filename)

print("파일 복사 및 이름 변경이 완료되었습니다.")

파일 복사 및 이름 변경이 완료되었습니다.


In [1]:
import json
import random
import glob

# 모든 json 파일을 읽어들이기
json_files = glob.glob('../data/train/train_*.json')

# 라벨별로 이미지 리스트 저장
label_image_dict = {}

# 각 json 파일 처리
for json_file in json_files:
    with open(json_file, 'r') as f:
        data = json.load(f)
        label = data['label']
        image_path = data['image_path']
        
        if label not in label_image_dict:
            label_image_dict[label] = []
        
        label_image_dict[label].append(image_path)

# 가장 많은 라벨의 이미지 수 찾기
max_count = max(len(images) for images in label_image_dict.values())

# 라벨별로 랜덤 샘플링
sampled_image_dict = {}

for label, images in label_image_dict.items():
    sampled_images = random.sample(images, min(max_count, len(images)))
    sampled_image_dict[label] = sampled_images


ValueError: max() arg is an empty sequence

In [ ]:
import shutil

for label, images in sampled_image_dict.items():
    for image in images:
        shutil.copy(image, f'../data_sample/sampled_train/{label}/')
